In [1]:
import torch 
from torch import nn

In [2]:
# convolution Block
class ConvolutionBlock(nn.Module):
    def __init__(self,in_channels,out_channels,kernel_size,stride,padding):
        super().__init__()
        self.Conv_0=nn.Conv2d(in_channels,out_channels,kernel_size,stride,padding)
        self.BatchNorm_0=nn.BatchNorm2d(out_channels)
    def forward(self,x):
        return self.BatchNorm_0(self.Conv_0(x))

In [3]:
# crating a bottle neck block
# here the output features are more then the input of the next layer
# so we will adjust it here
class BottleNeckBlock(nn.Module):
    def __init__(self,in_channel,out_channel,first=False):
        super().__init__()
        # the input feature are 256 and the input is 64 
        # so  we will divide the features and res channel are //4
        res_channel=in_channel//4
        stride=1

        self.projection = in_channel!=out_channel
        if self.projection:
            self.pro=ConvolutionBlock(in_channel,out_channel,1,2,0)
            stride=2
            res_channel=in_channel//2


        if first:
            self.p=ConvolutionBlock(in_channel,out_channel,1,1,0)
            stride=1
            res_channel=in_channel

        self.Conv_1=ConvolutionBlock(in_channel,res_channel,1,1,0)
        self.Conv_2=ConvolutionBlock(res_channel,res_channel,3,stride,1)
        self.Conv_3=ConvolutionBlock(res_channel,out_channel,1,1,0)
        self.relu=nn.ReLU()

    def forward(self,x):
        f=self.relu(self.Conv_1(x)) 
        f=self.relu(self.Conv_2(f))
        f=self.Conv_3(f)

        if self.projection:
            x=self.p(x)
        h=self.relu(torch.add(f,x))

        return h


In [4]:
# creating the resnet Block

class DriverClass(nn.Module):
    def __init__(self,no_Block,in_channel,classes=1000):
        super().__init__()
        out_features=[256,512,1024,2048]
        self.Block=nn.ModuleList([BottleNeckBlock(64,256,True)])

        for i in range(len(out_features)):
            if i>0:
                self.Block.append(BottleNeckBlock(out_features[i-1],out_features[i]))
            for _ in range(no_Block[i]-1):
                self.Block.append(BottleNeckBlock(out_features[i],out_features[i]))
        

        self.Conv=ConvolutionBlock(in_channel,64,7,2,3)
        self.maxPool=nn.MaxPool2d(kernel_size=3,stride=2,padding=1)
        self.avePool=nn.AdaptiveAvgPool2d((1,1))
        self.fc=nn.Linear(2048,classes)

        self.relu=nn.ReLU()


    def forward(self,x):
        x=self.relu(self.Conv(x))
        x=self.maxPool(x)
        for block in self.Block:
            x=block(x)

        x=self.avePool(x)
        x=torch.flatten(x,1)
        x=self.fc(x)

        return x
    

In [7]:
import torch 
import torch.nn as nn

# ConvBlock
class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride, padding):
        super().__init__()
        self.c = nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size, stride=stride, padding=padding)
        self.bn = nn.BatchNorm2d(out_channels)
    
    def forward(self, x):
        return self.bn(self.c(x))

# Bottleneck ResidualBlock 
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, first=False):
        super().__init__()
        res_channels = in_channels // 4
        stride = 1

        self.projection = in_channels!=out_channels
        if self.projection:
            self.p = ConvBlock(in_channels, out_channels, 1, 2, 0)
            stride = 2
            res_channels = in_channels // 2

        if first:
            self.p = ConvBlock(in_channels, out_channels, 1, 1, 0)
            stride = 1
            res_channels = in_channels


        self.c1 = ConvBlock(in_channels, res_channels, 1, 1, 0) 
        self.c2 = ConvBlock(res_channels, res_channels, 3, stride, 1)
        self.c3 = ConvBlock(res_channels, out_channels, 1, 1, 0)
        self.relu = nn.ReLU()

    def forward(self, x):
        f = self.relu(self.c1(x))
        f = self.relu(self.c2(f))
        f = self.c3(f)

        if self.projection:
            x = self.p(x)

        h = self.relu(torch.add(f, x))
        return h

# ResNetx
class ResNet(nn.Module):
    def __init__(
        self, 
        config_name : int, 
        in_channels=3, 
        classes=1000
        ):
        super().__init__()

        configurations = {
            50 : [3, 4, 6, 3],
            101 : [3, 4, 23, 3],
            152 : [3, 8, 36, 3]
        }

        no_blocks = configurations[config_name]

        out_features = [256, 512, 1024, 2048]
        self.blocks = nn.ModuleList([ResidualBlock(64, 256, True)])

        for i in range(len(out_features)):
            if i > 0:
                self.blocks.append(ResidualBlock(out_features[i-1], out_features[i]))
            for _ in range(no_blocks[i]-1):
                self.blocks.append(ResidualBlock(out_features[i], out_features[i]))
        
        self.conv1 = ConvBlock(in_channels, 64, 7, 2, 3)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.avgpool = nn.AdaptiveAvgPool2d((1,1))
        self.fc = nn.Linear(2048, classes)

        self.relu = nn.ReLU()

        # self.init_weight()

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.maxpool(x)
        for block in self.blocks:
            x = block(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x

    # def init_weight(self):
    #     for layer in self.modules():
    #         if isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear):
    #             nn.init.kaiming_normal_(layer.weight)


if __name__ == "__main__":
    config_name = 50 # 50-layer
    resnet50 = ResNet(50)
    image = torch.rand(1, 3, 224, 224)
    print(resnet50(image).shape)

torch.Size([1, 1000])
